# Location Viz Sample Notebook - Scala

In this notebook we will work with data from our Weighted Position model for Presence Insights to visualize location data before and after we apply our smoothing algorithms. 

### Load the data

First, we will need to upload our location data file to the object store.
- Create an object store in your Spark service instance
- On the palette to the right, navigate to the Data Sources tab
- Download [sample data](https://raw.githubusercontent.com/reinaldo422/bluemix-spark-viz/master/smoothed_data.json) (right-click and Save As...) and add as a source


In [1]:
//Import SQLContext and data types
import org.apache.spark.sql._
import org.apache.spark.sql.types._

// Spark Notebook has existing SparkContext (sc)
val sqlContext = new SQLContext(sc)
import sqlContext.implicits._

// Read sample Presence Insights location data
val json = sqlContext.read.json("swift://notebooks.spark/smoothed_data.json")

### Separate each individual event

In [2]:
import org.apache.spark.sql.functions.explode

val devicesDF = json.select(
    $"sites",
    explode($"data").alias("data")).select(
        $"data.original.device_descriptor".alias("device"),
        $"sites.floors.properties.z".getItem(0).alias("z"),
        $"data.original.x",
        $"data.original.y",
        $"data.smooth_x",
        $"data.smooth_y") 

devicesDF.registerTempTable("devices")


In [3]:
devicesDF.show()

### Select device and floor

In [4]:
val plot_data = sqlContext.sql("SELECT * FROM devices WHERE device='device1' AND z=0")

## Plotting with Brunel
[Brunel project](http://brunel.mybluemix.net/docs/) defines interactive data visualizations based on tabular data. It supports Spark dataframes and automatically chooses good transforms, mappings, and formatting for your data.

In [5]:
%AddJar -magic http://www.brunelvis.org/jar/spark-kernel-brunel-all-1.1.jar 

### Raw data

In [6]:
%%brunel data('plot_data') x(x) y(y)

### Smooth data

In [7]:
%%brunel data('plot_data') x(smooth_x) y(smooth_y)